# Two coupled TLS: the effect of decoherence on entanglement

The plan:
1. Start from 2 uncoupled qubits
2. Simulate and observe their entanglement
3. Arrive at a perfectly coupled state


In [ ]:
from qutip import bell_state, ket2dm

# Start from Bell state 00, namely (1/sqrt(2)) * (|00> + |11>)
psi = bell_state('00')

density_matrix = ket2dm(psi)

print(psi)

Quantum object: dims=[[2, 2], [1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.70710678]]
